In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Minimax Engine

Until now, a move was evaluated according to the value the heuristic gives to the resulting position. Thus, the chess engine lacks the foresight to find tactical combinations or a mate in several moves.
With the help of a search algorithm all future positions can be found, which are created after playing a certain number of moves. The below [game tree](https://www.yosenspace.com/posts/computer-science-game-trees.html) shows how the resulting positions in the game `TicTacToe` can be represented as a search tree. The root node is the initial position, while the leaf nodes contain the final positions that will be evaluated. A special feature of TicTacToe is that the leaf nodes are terminal states. In chess, on the other hand, the search usually has to be stopped at a certain depth, because the number of states is simply too large even for modern computers. The leaf nodes are therefore not necessarily terminal states and are thus evaluated with the already known heuristics. 

![TicTacToe GameTree](./images/tictactoe-gametree.jpg)

By limiting the search depth, however, the original problem is only partially solved. As before, it can happen that the considered final position ends, for example, in the middle of a slugfest and is correspondingly wrongly estimated. It also often leads to the fact that the engine tries to delay a foreseeable problem by giving chess etc. until it is no longer visible due to the search depth. 
This is called the [horizon effect](https://www.chessprogramming.org/Horizon_Effect), which can have a very strong impact on the engine's playing strength and is discussed accordingly in later chapters.

First, however, some search algorithms are presented, starting with the `MiniMax algorithm`. For this purpose, first a class `MiniMaxEngine` is defined,
which inherits from `Engine`,
but many parts will be familiar from the `EvaluationEngine`.
It expects an instance of an `Evaluator` to be passed at construction time as does the `EvaluationEngine`.
Additionally an integer `look_ahead_depth` is expected to specify the depth of the search tree.

In [ ]:
import random
import chess.engine
from converted_notebooks.s04_engine_interface import Engine, ScoredMove
from converted_notebooks.s08_evaluation import Evaluator


class MiniMaxEngine(Engine):

    def __init__(self, evaluator: Evaluator, look_ahead_depth):
        self.evaluator = evaluator
        self.look_ahead_depth = look_ahead_depth

The actual `MiniMax algorithm` is realized recursively with the function `_value`,
which takes the chess board `board` and the remaining depth `depth` of the current node of the search tree as parameters.
In each recursive pass, the method returns the value of the current node from the perspective of white.

First, the `Evaluators` method `evaluate` checks whether the current position is a terminal state. If this is the case, the evaluation of this from the perspective of white is returned accordingly. 
Otherwise, it is checked whether the maximum search depth has been reached and it is therefore a leaf node. In this case the `heuristic` method of the `Evaluator` is used and the value from the perspective of white is returned.

If neither of the termination conditions is true, the recursive part of the method is executed. For each possible move the new position is generated and recursively calculated with `_value`. In total, a list of values is generated for each child node in the search tree. 

Finally, the question remains how the node itself is evaluated based on the evaluations of the child nodes. The key idea of the `MiniMax algorithm` is that there is a minimizing and a maximizing player. When it is White's turn, he tries to maximize the valuation for his move. Accordingly, in this case the maximum value of the child nodes is returned. If, on the other hand, it is black's turn, he tries to minimize the valuation for his move. Therefore, the minimum value of the child nodes is returned. 
Overall, the algorithm assumes that both players want to win and play the best move for themselves. The respective position is thus evaluated as the future position that would be reached if both players played optimally.

In [ ]:
from chess.engine import PovScore, Score
import chess


def _value(self, board: chess.Board, depth: int) -> PovScore:
    if (score := self.evaluator.evaluate(board)) is not None:
        return score
    if depth == 0:
        return self.evaluator.heuristic(board)

    scores = []
    for move in board.legal_moves:
        board.push(move)
        score = self._value(board, depth - 1)
        score = score.white()
        scores.append(self._value(board, depth - 1).white())
        board.pop()

    if board.turn is chess.WHITE:
        return PovScore(max(scores), chess.WHITE)
    else:
        return PovScore(min(scores), chess.WHITE)


MiniMaxEngine._value = _value

To score a move
we define the `_evaluate_move` method
which takes the instance of the `MiniMaxEngine',
and a chess board `board`,
and a chess move `move`.
After making the move
it calls the recursive `_value` method to calculate the score
before restoring the board and returning a `ScoredMove` containing the move and its score.

In [ ]:
def _evaluate_move(self, board: chess.Board, move: chess.Move):
    board.push(move)
    score = self._value(board, self.look_ahead_depth - 1)
    board.pop()
    return ScoredMove(score=score.white(), move=move)


MiniMaxEngine._evaluate_move = _evaluate_move

The method `_evalutate_moves` is the same as before.

In [ ]:
def _evaluate_moves(self, board: chess.Board):
    return [self._evaluate_move(board, move) for move in board.legal_moves]


MiniMaxEngine._evaluate_moves = _evaluate_moves

The `analyse` method is not changed.

In [ ]:
def analyse(self, board: chess.Board) -> list[ScoredMove]:
    nextMoves = self._evaluate_moves(board.copy())
    random.shuffle(nextMoves)

    whitesTurn = board.turn is chess.WHITE
    nextMoves.sort(reverse=whitesTurn)

    return nextMoves


MiniMaxEngine.analyse = analyse

Neither is the `play` method.

In [ ]:
def play(self, board: chess.Board) -> chess.engine.PlayResult:
    bestScoredMove = self.analyse(board)[0]
    return chess.engine.PlayResult(move=bestScoredMove.move, ponder=None)


MiniMaxEngine.play = play

To demonstrate how the algorithm works, a special chess position is constructed in which only a limited number of moves are possible and the evaluation varies greatly depending on the move.

In [ ]:
sample_minimax_board = chess.Board("7k/7P/7P/8/8/p7/8/7K b - - 0 1")

In [ ]:
import IPython.display

IPython.display.display(sample_minimax_board)

An additional auxiliary class `MinMaxTree` was written, which however only serves to visualize the search tree and is therefore not explained in more detail. An instance of the class can be created with the method `add_tree_to_engine`. If then a position is analyzed with `analyze`, the tree can be drawn afterwards with `draw`. 

In [ ]:
import utils.min_max_tree
from converted_notebooks.s08_evaluation import standard_evaluator

random.seed(42)

engine = MiniMaxEngine(evaluator=standard_evaluator, look_ahead_depth=3)
tree = utils.min_max_tree.add_tree_to_engine(engine)
print(engine.analyse(sample_minimax_board))
tree.draw()

In the constructed example, the search tree can still be well surveyed. This is due in particular to the low branching factor in the first two moves. On average, however, the [branching factor is 35 - 38](https://www.chessprogramming.org/Branching_Factor). As a consequence, the number of nodes with higher depth quickly exceeds the computational capacity of modern computers. 

The following position, which arises from the [Exchange Variation of the Queen's Gambit Declined](https://en.wikipedia.org/wiki/Queen%27s_Gambit_Declined#Exchange_Variation:_4.cxd5_exd5), will demonstrate this.

In [ ]:
middlegame_board = chess.Board(
    "r1bqrnk1/pp2bppp/2p2n2/3p2B1/3P4/2NBPN2/PPQ2PPP/R4RK1 w - - 7 11"
)

In [ ]:
IPython.display.display(middlegame_board)
print(f"Number of moves: {len(list(middlegame_board.legal_moves))}")

If this position is evaluated with the `MiniMaxEngine` at depth three, it is immediately noticeable that the calculation is considerably slower. The evaluation of the tree also shows that it contains a total of over 70'000 nodes. At a depth of four, the number of nodes increases to 2'342'944. 

In [ ]:
import utils.min_max_tree
from converted_notebooks.s08_evaluation import standard_evaluator

random.seed(42)

engine = MiniMaxEngine(evaluator=standard_evaluator, look_ahead_depth=3)
tree = utils.min_max_tree.add_tree_to_engine(engine)
result_minimax = engine.analyse(middlegame_board)

In [ ]:
print(result_minimax)
tree.count()